In [1]:
import matplotlib.pyplot as plt
import numpy as np
from utils import reshape_data, get_dataset, get_dataloader, SeismicDataset
import torch
from torchvision import transforms
from networks import DemoNet
from torchsummary import summary

c:\Users\leons\AppData\Local\miniconda3\envs\leon_torch\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
net = DemoNet().to('cuda')  
summary(net, (1, 28, 28))

RuntimeError: mat1 and mat2 shapes cannot be multiplied (2x392 and 784x10)

In [ ]:
get_dataset()

In [ ]:
data_path = 'data/dataset.npy'
number_degradations = 6 # total 12
reshape_data(data_path, number_degradations)

In [ ]:
dataset = np.load(f'data/dataset_reshaped.npy', allow_pickle=True)
print(dataset.shape); print(len(dataset))

In [ ]:
idx = 12
fig, ax = plt.subplots(1,2, figsize=(10,5))
ax[0].imshow(dataset[idx][0], cmap='seismic')
ax[1].imshow(dataset[idx][1], cmap='seismic')
plt.show()


In [ ]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Resize((128, 128)),])
dataset = SeismicDataset(data_path='data/dataset_reshaped.npy', transform=transform)

In [ ]:
print(dataset.__len__())
fig, ax = plt.subplots(1,2, figsize=(10,5))
ax[0].imshow(dataset[112][0].squeeze(), cmap='seismic')
ax[1].imshow(dataset[112][1].squeeze(), cmap='seismic')

In [ ]:
trainloader, valoader, testloader = get_dataloader(batch_size=32, num_workers=1, data_path='data/dataset_reshaped.npy', im_size=(256,256))

In [ ]:
len(trainloader), len(valoader), len(testloader)

In [ ]:
img, degr = next(iter(trainloader))

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(10,5))
ax[0].imshow(img[-1].squeeze(), cmap='seismic')
ax[1].imshow(degr[-1].squeeze(), cmap='seismic')